In [7]:
import random
import equadratures as eq
import numpy as np
import matplotlib.pyplot as plt

from ModelTree import ModelTree
from linear_regr import linear_regr

from sklearn.metrics import mean_squared_error
import scipy.stats as st

In [8]:
def sample(N, f, sample_from):
    X, y = [], []
    for i in range(N):
        x = sample_from[0] + random.random() * (sample_from[1] - sample_from[0])
        X.append(np.array(x))
        y.append(np.array(f(x, True)))
    return np.array(X), np.array(y)

In [9]:
def evaluate(model, f, sample_from):
    X, y = sample(30, f, sample_from)
    X = np.reshape(X, (X.shape[0], 1))
    y = np.reshape(y, (y.shape[0], 1))

    return mean_squared_error(y, model.predict(X))

In [10]:
def plot(models, f, X_train, y_train):
    
    x_test = np.reshape(np.linspace(-.5, 1.5, 100), (100, 1))
    X_train = np.reshape(X_train, (X_train.shape[0], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], 1))

    fig = plt.figure(figsize=(16, 9), dpi= 50)
    ax = fig.add_subplot(1,1,1)
    
    
    
    plt.scatter(X_train[:,0], y_train[:,0], marker='o', s=80, color='blue', label='Training data')
    
    plt.plot(x_test, [f(x, False) for x in x_test], 'k-', label='True function', color = 'black')

    colours = ['green', 'red', 'purple']
    i = 0
    
    for model in models:
        model.fit(X_train,y_train)
        
        if model.__class__.__name__ == 'ModelTree':
            print(f"Function: {f.__name__}, Model: {model.model.__class__.__name__}, Error (MSE) in [-0.5,1.5]: {evaluate(model,f,[-0.5,1.5])}")
            plt.plot(x_test, model.predict(x_test), 'k-', label=str(model.model.__class__.__name__), color = colours[i])
        
        else:
            print(f"Function: {f.__name__}, Model: {model.__class__.__name__}, Error (MSE) in [-0.5,1.5]: {evaluate(model,f,[-0.5,1.5])}")
            plt.plot(x_test, model.predict(x_test), 'k-', label=str(model.__class__.__name__), color = colours[i])
        
        
        plt.xlabel('$X$', fontsize=13)
        plt.ylabel('$Y$', fontsize=13)
        plt.xticks(fontsize=13)
        plt.yticks(fontsize=13)
        i += 1
    
    ax.set_ylim([-3,4])

    plt.legend(prop={'size': 20})

In [11]:
class Poly:
    def __init__(self, order):
        self.order = order
        self.poly = False
        
    def fit(self, X, y):

        param = eq.Parameter(distribution='Uniform', lower=0, upper=1., order=self.order)
        myParameters = [param for i in range(1)] # one-line for loop for parameters
        
        myBasis = eq.Basis('Univariate')
        self.poly = eq.Poly(myParameters, myBasis, method='least-squares', sampling_args={'sample-points':X, 'sample-outputs':y} )
        self.poly.set_model()
        
    def predict(self, X):
        return self.poly.get_polyfit(np.array(X))

In [12]:
class LinTree:
    def __init__(self):
        self.poly = Poly(1)
    
    def fit(self,X,y):
        self.poly.fit(X,y)
        
    def predict(self, X):
        return self.poly.predict(X)
    
    def loss(self, X, y, y_pred):
        return mean_squared_error(y, y_pred)
    

In [13]:
class PolyTree:
    def __init__(self):
        self.poly = Poly(3)
    
    def fit(self,X,y):
        self.poly.fit(X,y)
        
    def predict(self, X):
        return self.poly.predict(X)
    
    def loss(self, X, y, y_pred):
        return mean_squared_error(y, y_pred)
    

In [14]:
def f_orig(x, noise):
    
    if noise:
        noise = 0.1 * st.norm.rvs(0, 1)
    else:
        noise = 0

    if x < 0.5:
        return 20*x**3 + x + 0.5 + noise
    elif x > 0.5:
        return -x**2 + x + noise

In [15]:
def f_osc(x, noise):
    
    if noise:
        noise = 0.1 * st.norm.rvs(0, 1)
    else:
        noise = 0
    if x < 0.5:
        return 25*(x-0.25)**2 - 1.0625 + noise
    elif x > 0.5:
        return 25*(x-0.75)**2 - 1.0625 + noise

In [16]:
def f_change(x, noise):
    if noise: 
        noise = 0.1 * st.norm.rvs(0, 1)
    else:
        noise = 0
    if x < 0.5:
        return 0.5 + noise
    else:
        return 25*(x-0.75)**2 - 1.0625 + noise

In [17]:
def f_anomaly(x, noise):
    if noise: 
        noise = 0.1 * st.norm.rvs(0, 1)
    else:
        noise = 0
    return 10*(x-0.5)**3 + noise

Below we train the models with $x \in [0,1]$ and evaluate the MSE of the model where $x \in [-0.5,1.5]$

In [ ]:
i = 0
samples = [30,15,30,20,20]
for f in [f_orig, f_anomaly, f_anomaly]:
    X, y = sample(samples[i], f, [0,1])
    
    if i == 3:
        X = np.append(X,0.5)
        y = np.append(y,3)
        
    X = np.reshape(X, (X.shape[0], 1))
    y = np.reshape(y, (y.shape[0], 1))

    poly = Poly(3)

    poly_model = PolyTree()
    poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                           search_type="greedy", n_search_grid=100)
    
    linear_model = LinTree()
    linear_regr_tree = ModelTree(linear_model, max_depth=3, min_samples_leaf=10,
                           search_type="greedy", n_search_grid=100)

    plot([poly, poly_model_tree], f, X, y)
    
    
    
    i+=1

# Regression using polynomial trees 
This blog post describes and justifies the implementation of a new regression method for Effective Quadratures (EQ). We hope to make it clear the cases in which polynomial regression trees would be well suited, as well as highlight some of the drawbacks associated with this method. 

Currently in EQ, global polynomial regresssion has been implemented. This method provides the ability to quickly generate a surrogate model that is robust to noise in the training data as demonstrated below. On the other hand it performs poorly with  functions such as:
 - Discontinuous functions
 - Polynomial functions with a order change

In order to justify these claims we use  [Anson Wong's model tree](https://github.com/ankonzoid/LearningX/tree/master/advanced_ML/model_tree) with the EQ Poly class as the model. It should be noted that for simplicity a few restrictions have been made. The main restriction made is that a fixed order of 3 is chosen for all polynomials in the following examples. Additionally all example datasets are 2 dimensional for the purpose of visualisation. 

A discontinuous function $f(x)$ with $x \in [0,1]$ is constructed from two polynomial functions with added noise. Using 30 training samples from the function we can see the PolyTree is able to identify the break between the functions, resulting in signifigantly less error when compared to the global Poly approximation

In [ ]:
X, y = sample(30, f_orig, [0,1])
X = np.reshape(X, (X.shape[0], 1))
y = np.reshape(y, (y.shape[0], 1))

poly = Poly(3)

poly_model = PolyTree()
poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

plot([poly, poly_model_tree], f_orig, X, y)


Another function $g(x)$ with $x \in [0,1]$ is constructed from two polynomial functions with added noise. The two polynomial functions have different orders and, along with the previous example, demonstrate the ability for the tree to adapt to functions with several functional forms.

In [ ]:
X, y = sample(30, f_change, [0,1])
X = np.reshape(X, (X.shape[0], 1))
y = np.reshape(y, (y.shape[0], 1))

poly = Poly(3)

poly_model = PolyTree()
poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

linear_model = LinTree()
linear_regr_tree = ModelTree(linear_model, max_depth=3, min_samples_leaf=10,
                           search_type="greedy", n_search_grid=100)

plot([poly, poly_model_tree], f_change, X, y)

One limitation of the polynomial tree method is that as the training data is divided, overfitting becomes more likely. A demonstration of this is seen below, where the global Poly method approximation is relatively uneffected by an anomaly at $(0,1)$ in comparison to the tree method, which is significantly skewed.

In [ ]:
X, y = sample(30, f_anomaly, [0,1])

X = np.reshape(X, (X.shape[0], 1))
y = np.reshape(y, (y.shape[0], 1))

poly = Poly(3)

poly_model = PolyTree()
poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

plot([poly, poly_model_tree], f_anomaly, X, y)

X = np.append(X,np.array([0]))
y = np.append(y,np.array([1]))

plot([poly, poly_model_tree], f_anomaly, X, y)


Another comparison to be made is one between a linear and polynomial regression tree. The main reason we would not want to implement a linear regression tree is that the functions we want to model are very rarely linear. Due to this linear model trees will require a greater depth and as mentioned in the previous example, a greater depth will generally result in more overfitting. In the following example we adjust the hyperparameters of the linear tree so that its model has roughly the same error as the polynomial model. This results in 7 separate linear models compared to the 2 polynomial models used. 

In [ ]:
X, y = sample(30, f_osc, [0,1])

X = np.reshape(X, (X.shape[0], 1))
y = np.reshape(y, (y.shape[0], 1))

poly = Poly(3)

poly_model = PolyTree()
poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

linear_model = LinTree()
linear_regr_tree = ModelTree(linear_model, max_depth=3, min_samples_leaf=1,
                       search_type="greedy", n_search_grid=100)

plot([poly, poly_model_tree, linear_regr_tree], f_osc, X, y)

# Conclusion and future plans
In this blog we have demonstrated the benefits of implementing an orthogonal polynomial regression tree class in EQ, as well as highlighted examples in which it would not be a suitable model to use. Over the coming weeks we will compare various approaches for deciding where to "split" the tree.

In [ ]:
def _fit_model(X, y, order):

	N, d = X.shape
	myParameters = []

	for d in range(d):
		d = [X[i,d] for i in range(N)]
		d_min = max(d)
		d_max = min(d)
		myParameters.append(Parameter(distribution='Uniform', lower=d_min, upper=d_max, order=order))

	myBasis = Basis('tensor-grid')
	poly = eq.Poly(myParameters, myBasis, method='least-squares', sampling_args={'sample-points':X, 'sample-outputs':y} )
	poly.set_model()

	return poly

In [ ]:
X, y = sample(30, f_osc, [0,1])
_fit_model(X,y,3)

In [21]:
import sys
sys.path.append('~/Effective-Quadratures/equadratures/')
from equadratures import polytree

In [30]:
import random
import numpy as np

def f(x1,x2):
    return 4 * x1 ** 3 + x1 ** 2 - 10 * x2 ** 2 + x2 + 1

def sample():
    X, y = [], []
    for i in range(100):
        x1, x2 = random.random(), random.random()        
        X.append(np.array([x1, x2]))
        y.append(np.array(f(x1, x2)))
    return np.array(X), np.array(y)

X, y = sample()
y = np.reshape(y, (y.shape[0], 1))

tree = polytree.PolyTree()
tree.fit(X, y)

for poly in tree.get_polys():
    print(poly.get_mean_and_variance())

(1.1237490265407237, 0.15483745141337188)
(0.052289968985634205, 0.08740227027500533)
(-3.626655544220043, 4.159178322825507)
(1.0388611472824902, 6.436435232942947)


In [23]:
X, y = sample(30, f_osc, [0,1])

X = np.reshape(X, (X.shape[0], 1))
y = np.reshape(y, (y.shape[0], 1))

poly = Poly(3)

poly_model = PolyTree()
poly_model_tree = ModelTree(poly_model, max_depth=3, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

polytree_model = polytree.PolyTree()
plot([poly, poly_model_tree, polytree_model], f_osc, X, y)

TypeError: sample() takes 0 positional arguments but 3 were given

In [31]:
print(str(tree.tree) + '\n\n')
tree.export_graphviz('tree.png',np.array(['a','b', 'c']), export_png=True, export_pdf=False)


{'name': 'node', 'index': 0, 'loss': 2.3303993068328602e-29, 'poly': <equadratures.poly.Poly object at 0x7f1b6f2727b8>, 'n_samples': 100, 'j_feature': 0, 'threshold': 0.5910402893468232, 'children': {'left': {'name': 'node', 'index': 1, 'loss': 8.49337405092615e-30, 'poly': <equadratures.poly.Poly object at 0x7f1b6f26c518>, 'n_samples': 67, 'j_feature': 1, 'threshold': 0.4616752092246966, 'children': {'left': {'name': 'node', 'index': 3, 'loss': 4.396954465270145e-30, 'poly': <equadratures.poly.Poly object at 0x7f1b6f275e80>, 'n_samples': 35, 'j_feature': 1, 'threshold': 0.3252303786902898, 'children': {'left': {'name': 'node', 'index': 5, 'loss': 8.506190587710292e-31, 'poly': <equadratures.poly.Poly object at 0x7f1b6f26fcc0>, 'data': (array([[0.17637184, 0.0997231 ],
       [0.4708996 , 0.19965024],
       [0.47481779, 0.23681339],
       [0.21524359, 0.19787065],
       [0.42355398, 0.01969794],
       [0.39664045, 0.21575758],
       [0.3484915 , 0.31755004],
       [0.11104213, 0.